The goal of this notebook is to try AI Search in Azure.

See: https://python.langchain.com/v0.2/docs/integrations/vectorstores/azuresearch/

In [2]:
import os
from dotenv import load_dotenv, find_dotenv